In [ ]:
import os
from dotenv import load_dotenv
from openai import OpenAI

In [ ]:
import gradio as gr 

In [ ]:
load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")

In [ ]:
openai = OpenAI()

In [ ]:
system_message = "You are a helpful assistant"

def message_gpt(prompt):
    messages = [{"role": "system", "content": system_message}, {"role": "user", "content": prompt}]
    response = openai.chat.completions.create(model="gpt-4.1-mini", messages=messages)
    return response.choices[0].message.content

In [ ]:
message_gpt("What is today's date?")

In [ ]:
def shout(text):
    print(f"Shout has beeb called with input: {text}")
    return text.upper()

In [ ]:
shout("hello")

In [ ]:
gr.Interface(fn=shout, inputs="textbox", outputs="textbox", flagging_mode="never").launch()

In [ ]:
message_input = gr.Textbox(label="Your message:", info="Enter a message to be shouted", lines=7)
message_output=gr.Textbox(label="Response:", lines=8)
view = gr.Interface(
    fn=shout,
    title="Shout",
    inputs=[message_input],
    outputs=[message_output],
    examples=["hello", "howdy"],
    flagging_mode="never"
)
view.launch()

In [ ]:
message_input = gr.Textbox(label="Your message:", info="Enter a message for GPT", lines=7)
message_output=gr.Textbox(label="Response:", lines=8)
view = gr.Interface(
    fn=message_gpt,
    title="GPT",
    inputs=[message_input],
    outputs=[message_output],
    examples=["hello", "howdy"],
    flagging_mode="never"
)
view.launch()

In [ ]:
system_message = "You are a helpful assistant that responds in markdown without code blocks"
message_input = gr.Textbox(label="Your message:", info="Enter a message for GPT-4.1-mini", lines=7)
message_output = gr.Markdown(label="Response:")
view = gr.Interface(
    fn=message_gpt,
    title="GPT", 
    inputs=[message_input], 
    outputs=[message_output], 
    examples=[
        "Explain the Transformer architecture to a layperson",
        "Explain the Transformer architecture to an aspiring AI engineer",
        ], 
    flagging_mode="never"
    )
view.launch()

In [ ]:
def stream_gpt(prompt):
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt}
      ]
    stream = openai.chat.completions.create(
        model='gpt-4.1-mini',
        messages=messages,
        stream=True
    )
    result = ""
    for chunk in stream:
        result += chunk.choices[0].delta.content or ""
        yield result

In [ ]:
message_input = gr.Textbox(label="Your message:", info="Enter a message for GPT-4.1-mini", lines=7)
message_output = gr.Markdown(label="Response:")

view = gr.Interface(
    fn=stream_gpt,
    title="GPT", 
    inputs=[message_input], 
    outputs=[message_output], 
    examples=[
        "Explain the Transformer architecture to a layperson",
        "Explain the Transformer architecture to an aspiring AI engineer",
        ], 
    flagging_mode="never"
    )
view.launch()

In [ ]:
from scraper import fetch_website_contents

In [ ]:
system_message = """
You are an assistant that analyzes the contents of a company website landing page
and creates a short brochure about the company for prospective customers, investors and recruits.
Respond in markdown without code blocks.
"""

In [ ]:
def stream_brochure(company_name, url):
    yield ""
    prompt = f"Please generate a company brochure for {company_name}. Here is their landing page:\n"
    prompt += fetch_website_contents(url)
    result = stream_gpt(prompt)
    
    yield from result

In [ ]:
name_input = gr.Textbox(label="Company name:")
url_input = gr.Textbox(label="Landing page URL including http:// or https://")
message_output = gr.Markdown(label="Response:")
view = gr.Interface(
    fn=stream_brochure,
    title="Brochure Generator", 
    inputs=[name_input, url_input], 
    outputs=[message_output], 
    examples=[
            ["Hugging Face", "https://huggingface.co"],
            ["Edward Donner", "https://edwarddonner.com"]
        ], 
    flagging_mode="never"
    )
view.launch()